<a href="https://colab.research.google.com/github/tmontaj/scripter/blob/main/Notebooks/wave2letter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import tensorflow as tf
import numpy as np

In [26]:
class FirstBlock(tf.keras.layers.Layer):
  def __init__(self, filters = 250, kernel_size = 48, strides = 2, **kwargs):
    super().__init__(**kwargs)
    self.conv = tf.keras.layers.Conv1D(filters= filters, kernel_size = kernel_size, strides = strides, padding='same', name = "first")
    self.batch_norm = tf.keras.layers.BatchNormalization()
    self.relu = tf.keras.layers.ReLU()

  def call(self, input_):
    conv = self.conv(input_)
    batch_norm = self.batch_norm(conv)
    relu = self.relu(batch_norm)
    return relu

In [27]:
class MidBlock(tf.keras.layers.Layer):
  def __init__(self, name, filters = 250, kernel_size = 7, **kwargs):
    super().__init__(**kwargs)
    self.conv = tf.keras.layers.Conv1D(filters = filters, kernel_size = kernel_size, padding='same', name = name)
    self.batch_norm = tf.keras.layers.BatchNormalization()
    self.relu = tf.keras.layers.ReLU()

  def call(self, input_):  
    conv = self.conv(input_)
    batch_norm = self.batch_norm(conv)
    relu = self.relu(batch_norm)
    return relu

In [31]:
class LastBlock(tf.keras.layers.Layer):
  def __init__(self, output_size = 40, **kwargs):
    super().__init__(**kwargs)
    self.conv1 = tf.keras.layers.Conv1D(filters = 2000, kernel_size = 32, padding='same', name = "mid8")
    self.batch_norm = tf.keras.layers.BatchNormalization()
    self.relu = tf.keras.layers.ReLU()
    self.conv2 = tf.keras.layers.Conv1D(filters = 2000, kernel_size = 1, padding='same' , name = "last1")
    self.conv3 = tf.keras.layers.Conv1D(filters = output_size, kernel_size = 1, padding='same', name = "last2")

  def call(self, input_):  
    conv1 = self.conv1(input_)
    batch_norm1 = self.batch_norm(conv1)
    relu1 = self.relu(batch_norm1)

    conv2 = self.conv1(relu1)
    batch_norm2 = self.batch_norm(conv2)
    relu2 = self.relu(batch_norm2)

    conv3 = self.conv1(relu2)
    batch_norm3 = self.batch_norm(conv3)
    relu3 = self.relu(batch_norm3)
    return relu3

In [32]:
class Wav2Let(tf.keras.Model):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.first_block = FirstBlock()
    self.mid_block1 = MidBlock(name = "mid1")
    self.mid_block2 = MidBlock(name = "mid2")
    self.mid_block3 = MidBlock(name = "mid3")
    self.mid_block4 = MidBlock(name = "mid4")
    self.mid_block5 = MidBlock(name = "mid5")
    self.mid_block6 = MidBlock(name = "mid6")
    self.mid_block7 = MidBlock(name = "mid7")
    self.last_block = LastBlock()
        
  def call(self, input_):
    first_block = self.first_block(input_)
    mid_block1 = self.mid_block1(first_block)
    mid_block2 = self.mid_block2(mid_block1)
    mid_block3 = self.mid_block3(mid_block2)
    mid_block4 = self.mid_block4(mid_block3)
    mid_block5 = self.mid_block5(mid_block4)
    mid_block6 = self.mid_block6(mid_block5)
    mid_block7 = self.mid_block7(mid_block6)
    last_block = self.last_block(mid_block7)
    return last_block 


In [37]:
model = Wav2Let()


In [45]:
x = np.ones((1,100,100))
x

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]])

In [46]:
y = model(x)
y

ValueError: ignored